# Create Trimesh Grid for McDonald Valley Problem

## Load Modules and Setup Notebook

In [ ]:
import os
import sys
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy

from flopy.utils.gridgen import Gridgen 

print(sys.version)
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

In [ ]:
datapath = '../data/mcdonaldvalley-unstructured'
ws = './data/ex06-trimesh'

## Create Base Grid

In [ ]:
nlay = 5
nrow = 40
ncol = 25
delr = 500.
delc = 500.
top = 100.
botm = np.zeros((nlay, nrow, ncol), dtype=np.float32)
botm[0] = -5.
botm[1] = -50
botm[2] = -51
botm[3] = -100
fname = os.path.join(datapath, 'bottom.csv')
botm[4] = np.loadtxt(fname, delimiter=',')
ms = flopy.modflow.Modflow()
dis = flopy.modflow.ModflowDis(ms, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr,
                               delc=delc, top=top, botm=botm)

## Read the Trimesh File into Python

In [ ]:
fname = os.path.join(datapath, 'TriMesh_with_densified_contours.exp')
with open(fname, 'r') as f:
    lines = f.readlines()

# read vertex info into a structured array
vtmp = [line.strip().split()[1:] for line in lines if line.startswith('N')]
dtype = [('ivert', int), ('x', float), ('y', float), ('label', '|S10'), ('ibound', int), ('elev', float)]
vertices = np.empty((len(vtmp)), dtype=dtype)
for i, line in enumerate(vtmp):
    for j, val in enumerate(line):
        vertices[i][j] = val

# read element info into a structured array
etmp = [line.strip().split()[1:] for line in lines if line.startswith('E')]
dtype = [('icell', int), ('iv1', int), ('iv2', int), ('iv3', int), ('label', '|S10'), ('ibound', int), ('elev', float)]
cells = np.empty((len(etmp)), dtype=dtype)
for i, line in enumerate(etmp):
    for j, val in enumerate(line):
        cells[i][j] = val

In [ ]:
verts = []
for ivert, x, y, label, ibound, elev in vertices:
    verts.append([x, y])
verts = np.array(verts, dtype=np.float)
    
iverts = []
for i, iv1, iv2, iv3, s, ibound, elev in list(cells):
    iverts.append([iv3 - 1, iv2 - 1, iv1 - 1])
    
ncpl = len(iverts)
nvert = verts.shape[0]

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mm = flopy.plot.PlotMapView(model=ms)
mm.plot_cvfd(verts, iverts, facecolor='none', edgecolor='k')
plt.xlim(verts[:, 0].min(), verts[:, 0].max())
plt.ylim(verts[:, 1].min(), verts[:, 1].max())

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mm = flopy.plot.PlotMapView(model=ms)
mm.plot_cvfd(verts, iverts, a=np.arange(len(iverts)), edgecolor='none', cmap='jet')
plt.xlim(verts[:, 0].min(), verts[:, 0].max())
plt.ylim(verts[:, 1].min(), verts[:, 1].max())

## Calculate Centroids

In [ ]:
# centroid and area array
from flopy.utils.cvfdutil import centroid_of_polygon, area_of_polygon
xcyc = np.empty((ncpl, 2), dtype=np.float)
area = np.empty((ncpl), dtype=np.float)
for icell in range(ncpl):
    vlist = [(verts[ivert, 0], verts[ivert, 1]) for ivert in iverts[icell]]
    xcyc[icell, 0], xcyc[icell, 1] = centroid_of_polygon(vlist)
    area[icell] = abs(area_of_polygon(*zip(*vlist)))
    
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mm = flopy.plot.PlotMapView(model=ms)
mm.plot_cvfd(verts, iverts, facecolor='none', edgecolor='k')
plt.plot(xcyc[:, 0], xcyc[:, 1], 'ko')
plt.xlim(verts[:, 0].min(), verts[:, 0].max())
plt.ylim(verts[:, 1].min(), verts[:, 1].max())

## Create DISV Arrays

In [ ]:
fname = os.path.join(datapath, 'bottom.csv')
bot = np.loadtxt(fname, delimiter=',')
bot_tm = ms.sr.interpolate(bot, xcyc)

vertices = []
for i in range(nvert):
    vertices.append((i, verts[i, 0], verts[i, 1]))

cell2d = []
for i in range(ncpl):
    cell2d.append([i, xcyc[i, 0], xcyc[i, 1], 3] + [iv for iv in iverts[i]])    

## Intersect the Grid with the Property Arrays

In [ ]:
fname = os.path.join(datapath, 'k_aq.csv')
kaq = np.loadtxt(fname, delimiter=',')
kaq_tm = ms.sr.interpolate(kaq, xcyc)

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mm = flopy.plot.PlotMapView(model=ms)
mm.plot_cvfd(verts, iverts, a=kaq_tm, edgecolor='none', cmap='jet')
plt.xlim(verts[:, 0].min(), verts[:, 0].max())
plt.ylim(verts[:, 1].min(), verts[:, 1].max())

In [ ]:
from flopy.utils.gridgen import read1d
kclay = np.empty((dis.nrow * dis.ncol), dtype=np.float)
fname = os.path.join(datapath, 'k_clay.txt')
f = open(fname, 'r')
kclay = read1d(f, kclay).reshape((dis.nrow, dis.ncol))
kclay_tm = ms.sr.interpolate(kclay, xcyc)

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mm = flopy.plot.PlotMapView(model=ms)
mm.plot_cvfd(verts, iverts, a=kclay_tm, edgecolor='none', cmap='jet')
plt.xlim(verts[:, 0].min(), verts[:, 0].max())
plt.ylim(verts[:, 1].min(), verts[:, 1].max())

## Intersect the Features with the Grid

In [ ]:
ibd = cells['ibound']
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mm = flopy.plot.PlotMapView(model=ms)
mm.plot_cvfd(verts, iverts, a=ibd, edgecolor='none', cmap='jet')
plt.xlim(verts[:, 0].min(), verts[:, 0].max())
plt.ylim(verts[:, 1].min(), verts[:, 1].max())

In [ ]:
iriv = 1
iwel = 2
ilak = 3

idx = cells['ibound'] == iriv

rbot = -2.
ct = 0.
at = 0.

# cond_scale is needed to make sure the total conductance
# is similar to what is in the regular grid.
cond_scale = 10.28
rivspd = []
for ip1, iv1, iv2, iv3, txt, ibound, elev in cells[idx]:
    icell = ip1 - 1
    distance = xcyc[icell, 1]
    stage = 0. + distance / (18 * delc)
    cond = 1.e5
    cond = cond / (delc * delr) * area[icell]
    cond = cond * cond_scale
    ct += cond
    at += area[icell]
    rivspd.append([(0, icell), stage, cond, rbot])

In [ ]:
iriv = 1
iwel = 2
ilak = 3
idx = cells['ibound'] == ilak
chdspd = [[(0, ip1 - 1), 11.] for ip1, iv1, iv2, iv3, txt, ibound, elev in cells[idx]]

## Build the Model

In [ ]:
name = 'mv'
exe_name = os.path.abspath('../bin/mf6')
sim = flopy.mf6.MFSimulation(sim_ws=ws, sim_name=name, exe_name=exe_name)
tdis = flopy.mf6.ModflowTdis(sim)
ims = flopy.mf6.ModflowIms(sim, complexity='complex')
gwf = flopy.mf6.ModflowGwf(sim, save_flows=True)
dis = flopy.mf6.ModflowGwfdisv(gwf, nlay=5, ncpl=ncpl, nvert=nvert, 
                               top=cells['elev'],
                               botm=[-5, -50, -51, -100., bot_tm],
                               vertices=vertices, cell2d=cell2d)
ic = flopy.mf6.ModflowGwfic(gwf, strt=11.)
npf = flopy.mf6.ModflowGwfnpf(gwf, 
                              xt3doptions=True, save_specific_discharge=True,
                              icelltype=[1, 0, 0, 0, 0],
                              k=[kaq_tm, kaq_tm, kclay_tm, kaq_tm, kaq_tm],
                              k33=[0.25 * kaq_tm, 0.25 * kaq_tm, kclay_tm, 0.25 * kaq_tm, 0.25 * kaq_tm])
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chdspd)
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=.003641)
riv = flopy.mf6.ModflowGwfriv(gwf, stress_period_data=rivspd)
oc = flopy.mf6.ModflowGwfoc(gwf, 
                            head_filerecord=name + '.hds', 
                            budget_filerecord=name + '.bud',
                            saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
                            printrecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')])
sim.write_simulation()
sim.run_simulation()

## Post-Process the Results

In [ ]:
from flopy.utils.binaryfile import HeadFile, CellBudgetFile
fname = os.path.join(ws, name + '.hds')
headobj = HeadFile(fname)
head = headobj.get_data()[:, 0, :]
fname = os.path.join(ws, name + '.bud')
budobj = CellBudgetFile(fname, precision='double')
spdis = budobj.get_data(text='DATA-SPDIS')

ilay = 0

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 2, 1, aspect='equal')
pmv = flopy.plot.PlotMapView(gwf, ax=ax, layer=ilay)
pmv.plot_array(head, cmap='jet')
#pmv.plot_grid()

ax = fig.add_subplot(1, 2, 2, aspect='equal')
pmv = flopy.plot.PlotMapView(gwf, ax=ax, layer=ilay)
pmv.contour_array(head, levels=np.arange(20))
pmv.plot_specific_discharge(spdis)